In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn

from sklearn.svm import LinearSVR

# from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import chess

import glob
import joblib

In [3]:
def get_current_version():
    from pathlib import Path
    import pickle

    model_file_path = 'model_SVR_1.joblib'
    counter = 1

    while Path(model_file_path).is_file(): # ensure that no files are overwritten
        counter += 1
        model_file_path = f'model_SVR_{counter}.joblib'
    
    return counter

MODEL_NUMBER = get_current_version()
MODEL_NUMBER

1

In [4]:
X_loaded = joblib.load('data_X.joblib')
y_loaded = joblib.load('data_y.joblib')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_loaded, y_loaded, test_size = 0.2, random_state = 0)

In [6]:
X_train.shape, y_train.shape

((640000, 74), (640000,))

In [7]:
lin_svr = LinearSVR(verbose = 1)
lin_svr.fit(X_train, y_train)

c:\Users\kyle\Documents\LocalPrograms\Chess_Bot\.venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[LibLinear]

c:\Users\kyle\Documents\LocalPrograms\Chess_Bot\.venv\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR(verbose=1)

In [8]:
y_pred = lin_svr.predict(X_test)
np.sum(np.abs(y_pred - y_test)) / len(y_test)

2.5767398721077925

In [9]:
import joblib

filename = f'model_SVR_{2}.joblib'

joblib.dump(lin_svr, filename, compress=9)

['model_SVR_2.joblib']

In [10]:
model_loaded = joblib.load('model_SVR_2.joblib')

In [11]:
def predict_SVR(model, fen, move_number = 5, stochastic = True):

    board = chess.Board(fen)
    legal_moves_list = list(board.legal_moves)
    evals_list = []
    
    for move in legal_moves_list:
        
        # is_capture = board.is_capture(move)

        board.push(move)
        fen_array = fen_str_to_1d_array(board.fen())
        # print(fen_array.shape)

        pieces_counts = get_number_of_pieces(board.fen())

        inputs = np.concatenate((fen_array, pieces_counts))

        inputs = inputs.reshape(1, -1)

        eval_prediction = model.predict(inputs)

        evals_list.append(eval_prediction)

        if board.is_checkmate():
            return move # Always make a move which gives checkmate if possible.

        board.pop()

        # New portion (added 2024-04-09)
        if board.is_capture(move):
            if board.turn:
                evals_list[-1] += 0.5 # Modify to add piece value eventually
            else:
                evals_list[-1] -= 0.5 # Modify to add piece value eventually
    

    evals_list = np.array(evals_list)
    # print(evals_list)
    # print(np.array(legal_moves_list))

    sorted_indices = np.argsort(evals_list)
    
    # print(sorted_indices)

    if board.turn:
        '''
        if it's white's turn, we must reverse the array such that the highest evaluation is first
        if it's black's turn, keep the array ascending such that the lowest evaluation for the white pieces is first
        ''' 
        sorted_indices = sorted_indices[::-1]
    
    # print(np.array(legal_moves_list).shape)

    # Use the sorted indices to sort legal_moves and evals_list
    sorted_legal_moves = np.array(legal_moves_list)[sorted_indices]
    sorted_evals_list = evals_list[sorted_indices]

    if not stochastic: # if not using stochastic mode return best move
        return sorted_legal_moves[0]

    sample = np.random.random_sample()

    # print(sample)
    # print(sorted_legal_moves)

    if sample <= 0.65 or move_number > 7: # 65% chance for best move
        # print(f'playing best move')
        return sorted_legal_moves[0]
    elif sample <= 0.85 or move_number > 5: # 25% chance for second-best move
        return sorted_legal_moves[1]
    elif sample <= 0.975 or move_number > 3: #  7.5% chance for third-best move
        return sorted_legal_moves[2]
    else: # 2.5% chance for fourth-best move
        return sorted_legal_moves[3]

In [12]:
def fen_str_to_1d_array(fen):
    """
    Converts a FEN string representation of a chess board to a flat tensor representation.

    Args:
        fen (str): The FEN string representing the chess board.

    Returns:
        torch.Tensor: A flat tensor representation of the chess board.

    Example:
        >>> fen_str_to_flat_tensor('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
        tensor([[ -4.,  -2.,  -3.,  -5.,  -6.,  -3.,  -2.,  -4.],
                [ -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.],
                [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
                [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
                [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
                [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
                [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
                [  4.,   2.,   3.,   5.,   6.,   3.,   2.,   4.]])
    """    
    # Define a mapping from pieces to integers
    piece_to_int = {
        'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
        'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6,
    }

    # Split the FEN string into parts ## 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
    parts = fen.split(' ')
    ranks = parts[0].split('/') # Only process the board position (the first part)

    # Convert the ranks to a list of integers
    board = []
    for rank in ranks:
        for char in rank:
            if char.isdigit():
                # If the character is a digit, add that many zeros to the board
                board.extend([0] * int(char))
            else:
                # Otherwise, add the integer representation of the piece to the board
                board.append(piece_to_int[char])

    # Convert the board to a tensor
    board_array = np.array(board, dtype='float32')

    return board_array

In [13]:
def get_number_of_pieces(fen_str):
    """
    Get the number of pieces of a given type and color on the board.

    Parameters
    ----------
    board : chess.Board
        The chess board.

    Returns
    -------
    list of int, length 10
        Number of pieces on the board (not including kings)
        [white_pawns, white_knights, ... , black_rooks, black_queens]
    """
    
    piece_counts = []
    max_starting_pieces = [8, 2, 2, 2, 1]
    values = [1, 3, 3.1, 5, 9]

    board = chess.Board(fen_str)

    for j, color in enumerate([chess.WHITE, chess.BLACK]): # 0 for white, 1 for black
        for i, piece in enumerate([chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]):
            piece_counts.append(len(board.pieces(piece, color)) / max_starting_pieces[i] * values[i] * (-1)**j / 2) # White pieces are positive, black pieces are negative
    
    return np.array(piece_counts)

In [14]:
board = chess.Board()

import time

counter = 1
# try:
while True:
    counter += 1

    time.sleep(1)
    move = predict_SVR(model_loaded, board.fen(), move_number=counter)
    print(move)
    board.push(move[0])
    print(board)#.unicode()
# except:
#     pass

[Move.from_uci('g1h3')]
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
[Move.from_uci('g8h6')]
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
[Move.from_uci('h3g5')]
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
[Move.from_uci('h8g8')]
r n b q k b r .
p p p p p p p p
. . . . . . . n
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
[Move.from_uci('g5h7')]
r n b q k b r .
p p p p p p p N
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
[Move.from_uci('g8h8')]
r n b q k b . r
p p p p p p p N
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
[Move.from_uci('h7f8')]
r n b q k N . r
p p p p p p p .
. . . . . . . n
. . . . . . . .


KeyboardInterrupt: 